In [2]:
# Install the required modules
!pip install transformers
!pip install pandas
!pip install numpy
!pip install tqdm
!pip install sklearn

from google.colab import drive
drive.mount('/content/gdrive')

# !nvidia-smi

     |████████████████████████████████| 317kB 4.9MB/s 
     |████████████████████████████████| 860kB 65.4MB/s 
     |████████████████████████████████| 645kB 54.8MB/s 
     |████████████████████████████████| 1.0MB 49.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=396b4fc6e035182460a3e5f115f0e51da17dff56fc2e075b58c2c4dba09b1470
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses


In [0]:
import pandas as pd
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix
import numpy as np
import torch
from transformers import *

In [0]:
# Embeddings can be derived from the last 1 or 4 layers, to reduce the computational cost, we used only the last layer.

class Embeddings:
    LAST_LAYER = 1
    LAST_4_LAYERS = 2
    def __init__(self):
        self._tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self._bert_model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)
        self._bert_model.eval()

    def tokenize(self, sentence):
        """

        :param sentence: input sentence ['str']
        :return: tokenized sentence based on word piece model ['List']
        """
        marked_sentence = "[CLS] " + sentence + " [SEP]"
        tokenized_text = self._tokenizer.tokenize(marked_sentence)
        return tokenized_text

    def get_bert_embeddings(self, sentence):
        """

        :param sentence: input sentence ['str']
        :return: BERT pre-trained hidden states (list of torch tensors) ['List']
        """
        # Predict hidden states features for each layer

        tokenized_text = self.tokenize(sentence)
        indexed_tokens = self._tokenizer.convert_tokens_to_ids(tokenized_text)

        segments_ids = [1] * len(tokenized_text)

        # Convert inputs to PyTorch tensors
        tokens_tensor = torch.tensor([indexed_tokens])
        segments_tensors = torch.tensor([segments_ids])

        with torch.no_grad():
            encoded_layers = self._bert_model(tokens_tensor, segments_tensors)

        return encoded_layers[-1][0:12]

    def sentence2vec(self, sentence, layers):
        """

        :param sentence: input sentence ['str']
        :param layers: parameter to decide how word embeddings are obtained ['str]
            1. 'last' : last hidden state used to obtain word embeddings for sentence tokens
            2. 'last_4' : last 4 hidden states used to obtain word embeddings for sentence tokens

        :return: sentence vector [List]
        """
        encoded_layers = self.get_bert_embeddings(sentence)
        
        if layers == 1:
            # using the last layer embeddings
            token_embeddings = encoded_layers[-1]
            # summing the last layer vectors for each token
            sentence_embedding = torch.mean(token_embeddings, 1)
            return sentence_embedding.view(-1).tolist()

        elif layers == 2:
            token_embeddings = []
            tokenized_text = self.tokenize(sentence)

            batch_i = 0
            # For each token in the sentence...
            for token_i in range(len(tokenized_text)):

                # Holds 12 layers of hidden states for each token
                hidden_layers = []

                # For each of the 12 layers...
                for layer_i in range(len(encoded_layers)):
                    # Lookup the vector for `token_i` in `layer_i`
                    vec = encoded_layers[layer_i][batch_i][token_i]

                    hidden_layers.append(list(vec.numpy()))

                token_embeddings.append(hidden_layers)

            # using the last 4 layer embeddings
            token_vecs_sum = []

            # For each token in the sentence...
            for token in token_embeddings:
                # Sum the vectors from the last four layers.
                sum_vec = np.sum(token[-4:], axis=0)

                # Use `sum_vec` to represent `token`.
                token_vecs_sum.append(list(sum_vec))

            # summing the last layer vectors for each token
            sentence_embedding = np.mean(token_vecs_sum, axis=0)
            return sentence_embedding.ravel().tolist()

In [0]:
# Dataset: 3000 chunks * 3 authors

url = 'https://raw.githubusercontent.com/fy164251/text_style_transfer/master/Datasets/raw_text_3000.csv'
df = pd.read_csv(url)

X = df.text.astype('str')
y = df.author.astype('category')

lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(y.values)

In [0]:
model = Embeddings()

X_text = []
for sentence in tqdm(X):
    X_text.append(model.sentence2vec(sentence, layers=model.LAST_LAYER))


100%|██████████| 9000/9000 [35:17<00:00,  3.84it/s]


In [22]:
# X_df = pd.DataFrame(X_text)
# X_df.to_csv('./gdrive/My Drive/DL/Style/DistilBert_Embedding_3000.csv')

X_df = pd.read_csv('./gdrive/My Drive/DL/Style/DistilBert_Embedding_3000.csv')

X_train, X_val, y_train, y_val = train_test_split(X_df, y, stratify=y, random_state=1, test_size=0.2, shuffle=True)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=1, shuffle=True)

print(X_train.shape, X_val.shape)

(7200, 769) (900, 769)
